In [24]:
import pandas as pd
import numpy as np

In [25]:
# load data
df = pd.read_csv("../data/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv")
df.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# define to check if close price increased compare to the previous close
df["Trend"] = df.apply(lambda x: 1 if x["Btc_diff"] > 0 else (2 if x["Btc_diff"] <0 else 0), axis=1)

In [4]:
df[["Date", "Trend"]]

,Date,Trend
0,2012-01-03,0
1,2012-01-04,1
2,2012-01-05,1
3,2012-01-06,1
4,2012-01-09,0
...,...,...
2318,2021-03-25,2
2319,2021-03-26,1
2320,2021-03-29,1
2321,2021-03-30,1


In [5]:
ind_train_start = 360
ind_test_start = int((len(df) - 360) * 0.8)

In [6]:
df.columns

Index(['Date', 'Close', 'ma_5', 'ma_10', 'ma_20', 'ma_30', 'ma_60', 'ma_90',
       'ma_180', 'ma_240', 'ma_360', 'Gold', 'Nasdaq_Comp', 'SP500', 'INDU',
       'Oil', 'Btc_diff', 'Gold_diff', 'Nasdaq_diff', 'SP500_diff',
       'INDU_diff', 'Oil_diff', 'Btc_diffpct', 'Gold_diffpct',
       'Nasdaq_diffpct', 'SP500_diffpct', 'INDU_diffpct', 'Oil_diffpct',
       'Trend'],
      dtype='object')

In [7]:
# change here
X = df[['Close','ma_5', 'ma_10', 'ma_20', 'ma_30', 'ma_60', 'ma_90',
       'ma_180', 'ma_240', 'ma_360', 'Gold', 'Nasdaq_Comp', 'SP500', 'INDU', 'Oil']]
y = df[["Trend"]]

In [8]:
X_train = X.iloc[ind_train_start:ind_test_start]
X_test = X.iloc[ind_test_start:]
X_test = X_test.iloc[:len(X_test)-1]
X_test.iloc[0]

# for using close price
#y_train = y.iloc[ind_train_start + 1:ind_test_start + 1]
#y_test = y.iloc[ind_test_start+1:]
# for using other rows
y_train = y.iloc[ind_train_start+1:ind_test_start+1]
y_test = y.iloc[ind_test_start+1:]

In [9]:
print(X_train.shape, y_train.shape)

(1210, 15) (1210, 1)


In [10]:
ind_test = ind_test_start + 3
ind_test2 = ind_test - ind_test_start
print(df[["Date", "Close", 'Gold', 'Nasdaq_Comp', 'SP500', 'INDU', 'Oil', "Btc_diff", "Trend"]].iloc[ind_test])
print(X_test.iloc[ind_test2])
print(y_test.iloc[ind_test2])

Date           2018-04-10
Close              6880.0
Gold                12.77
Nasdaq_Comp        7094.3
SP500             2656.87
INDU              24408.0
Oil               18.5393
Btc_diff          -289.99
Trend                   2
Name: 1573, dtype: object
Close           6880.000000
ma_5            7010.556000
ma_10           7104.993000
ma_20           7699.797500
ma_30           8111.550333
ma_60           9297.584667
ma_90          10013.992444
ma_180         10563.570444
ma_240          9012.275833
ma_360          6798.611028
Gold              12.770000
Nasdaq_Comp     7094.300000
SP500           2656.870000
INDU           24408.000000
Oil               18.539300
Name: 1573, dtype: float64
Trend    1
Name: 1574, dtype: int64


In [11]:
from sklearn.preprocessing import StandardScaler

In [12]:
X_scaler = StandardScaler().fit(X)

In [13]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# for predict price
from tensorflow.keras.utils import to_categorical

In [15]:
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [16]:
print(X_train_scaled, y_train_categorical)

[[-0.54605436 -0.54795302 -0.55064387 ... -1.08271874 -0.99454033
   1.23129059]
 [-0.54570171 -0.54814733 -0.55085914 ... -1.10414619 -1.01866292
   1.23129059]
 [-0.54600382 -0.5482433  -0.55105954 ... -1.06661273 -0.98425508
   1.29250421]
 ...
 [ 0.27663567  0.31041562  0.39153495 ...  0.42200447  0.6268557
  -0.50653828]
 [ 0.32356145  0.29956747  0.37338922 ...  0.47328235  0.70089772
  -0.50653828]
 [ 0.31418547  0.30308153  0.35827909 ...  0.5208919   0.7448355
  -0.49627758]] [[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [17]:
print(X_train_scaled.shape, y_train_categorical.shape)

(1210, 15) (1210, 3)


In [18]:
from tensorflow.keras.models import Sequential
model = Sequential()

In [19]:
from tensorflow.keras.layers import Dense
number_inputs = len(X.columns)
model.add(Dense(units=30, activation='relu', input_dim=number_inputs))
model.add(Dense(units=30, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                480       
_________________________________________________________________
dense_1 (Dense)              (None, 30)                930       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 93        
Total params: 1,503
Trainable params: 1,503
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

In [22]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=1000,
    shuffle=True,
    verbose=2
)

Epoch 1/1000
38/38 - 0s - loss: 0.8959 - accuracy: 0.5190
Epoch 2/1000
38/38 - 0s - loss: 0.7552 - accuracy: 0.5686
Epoch 3/1000
38/38 - 0s - loss: 0.7184 - accuracy: 0.5793
Epoch 4/1000
38/38 - 0s - loss: 0.7086 - accuracy: 0.5868
Epoch 5/1000
38/38 - 0s - loss: 0.7078 - accuracy: 0.5760
Epoch 6/1000
38/38 - 0s - loss: 0.7018 - accuracy: 0.5876
Epoch 7/1000
38/38 - 0s - loss: 0.6992 - accuracy: 0.5975
Epoch 8/1000
38/38 - 0s - loss: 0.7023 - accuracy: 0.5917
Epoch 9/1000
38/38 - 0s - loss: 0.7008 - accuracy: 0.5917
Epoch 10/1000
38/38 - 0s - loss: 0.6977 - accuracy: 0.5967
Epoch 11/1000
38/38 - 0s - loss: 0.6991 - accuracy: 0.5909
Epoch 12/1000
38/38 - 0s - loss: 0.6950 - accuracy: 0.5967
Epoch 13/1000
38/38 - 0s - loss: 0.6957 - accuracy: 0.5934
Epoch 14/1000
38/38 - 0s - loss: 0.6951 - accuracy: 0.5992
Epoch 15/1000
38/38 - 0s - loss: 0.6939 - accuracy: 0.5950
Epoch 16/1000
38/38 - 0s - loss: 0.6925 - accuracy: 0.6050
Epoch 17/1000
38/38 - 0s - loss: 0.6962 - accuracy: 0.5967
Epoch 

Epoch 140/1000
38/38 - 0s - loss: 0.6648 - accuracy: 0.6421
Epoch 141/1000
38/38 - 0s - loss: 0.6648 - accuracy: 0.6339
Epoch 142/1000
38/38 - 0s - loss: 0.6678 - accuracy: 0.6264
Epoch 143/1000
38/38 - 0s - loss: 0.6643 - accuracy: 0.6306
Epoch 144/1000
38/38 - 0s - loss: 0.6648 - accuracy: 0.6306
Epoch 145/1000
38/38 - 0s - loss: 0.6628 - accuracy: 0.6347
Epoch 146/1000
38/38 - 0s - loss: 0.6688 - accuracy: 0.6314
Epoch 147/1000
38/38 - 0s - loss: 0.6643 - accuracy: 0.6372
Epoch 148/1000
38/38 - 0s - loss: 0.6675 - accuracy: 0.6231
Epoch 149/1000
38/38 - 0s - loss: 0.6640 - accuracy: 0.6347
Epoch 150/1000
38/38 - 0s - loss: 0.6666 - accuracy: 0.6355
Epoch 151/1000
38/38 - 0s - loss: 0.6631 - accuracy: 0.6306
Epoch 152/1000
38/38 - 0s - loss: 0.6661 - accuracy: 0.6331
Epoch 153/1000
38/38 - 0s - loss: 0.6645 - accuracy: 0.6298
Epoch 154/1000
38/38 - 0s - loss: 0.6651 - accuracy: 0.6413
Epoch 155/1000
38/38 - 0s - loss: 0.6625 - accuracy: 0.6347
Epoch 156/1000
38/38 - 0s - loss: 0.6626

Epoch 277/1000
38/38 - 0s - loss: 0.6577 - accuracy: 0.6281
Epoch 278/1000
38/38 - 0s - loss: 0.6527 - accuracy: 0.6421
Epoch 279/1000
38/38 - 0s - loss: 0.6573 - accuracy: 0.6331
Epoch 280/1000
38/38 - 0s - loss: 0.6552 - accuracy: 0.6372
Epoch 281/1000
38/38 - 0s - loss: 0.6540 - accuracy: 0.6446
Epoch 282/1000
38/38 - 0s - loss: 0.6568 - accuracy: 0.6347
Epoch 283/1000
38/38 - 0s - loss: 0.6527 - accuracy: 0.6355
Epoch 284/1000
38/38 - 0s - loss: 0.6554 - accuracy: 0.6380
Epoch 285/1000
38/38 - 0s - loss: 0.6502 - accuracy: 0.6405
Epoch 286/1000
38/38 - 0s - loss: 0.6536 - accuracy: 0.6380
Epoch 287/1000
38/38 - 0s - loss: 0.6546 - accuracy: 0.6364
Epoch 288/1000
38/38 - 0s - loss: 0.6524 - accuracy: 0.6339
Epoch 289/1000
38/38 - 0s - loss: 0.6539 - accuracy: 0.6405
Epoch 290/1000
38/38 - 0s - loss: 0.6527 - accuracy: 0.6438
Epoch 291/1000
38/38 - 0s - loss: 0.6530 - accuracy: 0.6438
Epoch 292/1000
38/38 - 0s - loss: 0.6519 - accuracy: 0.6306
Epoch 293/1000
38/38 - 0s - loss: 0.6541

Epoch 414/1000
38/38 - 0s - loss: 0.6439 - accuracy: 0.6479
Epoch 415/1000
38/38 - 0s - loss: 0.6447 - accuracy: 0.6471
Epoch 416/1000
38/38 - 0s - loss: 0.6420 - accuracy: 0.6471
Epoch 417/1000
38/38 - 0s - loss: 0.6439 - accuracy: 0.6463
Epoch 418/1000
38/38 - 0s - loss: 0.6416 - accuracy: 0.6413
Epoch 419/1000
38/38 - 0s - loss: 0.6414 - accuracy: 0.6471
Epoch 420/1000
38/38 - 0s - loss: 0.6459 - accuracy: 0.6446
Epoch 421/1000
38/38 - 0s - loss: 0.6448 - accuracy: 0.6380
Epoch 422/1000
38/38 - 0s - loss: 0.6433 - accuracy: 0.6446
Epoch 423/1000
38/38 - 0s - loss: 0.6436 - accuracy: 0.6496
Epoch 424/1000
38/38 - 0s - loss: 0.6424 - accuracy: 0.6496
Epoch 425/1000
38/38 - 0s - loss: 0.6464 - accuracy: 0.6455
Epoch 426/1000
38/38 - 0s - loss: 0.6453 - accuracy: 0.6331
Epoch 427/1000
38/38 - 0s - loss: 0.6428 - accuracy: 0.6512
Epoch 428/1000
38/38 - 0s - loss: 0.6439 - accuracy: 0.6421
Epoch 429/1000
38/38 - 0s - loss: 0.6449 - accuracy: 0.6455
Epoch 430/1000
38/38 - 0s - loss: 0.6416

Epoch 551/1000
38/38 - 0s - loss: 0.6405 - accuracy: 0.6488
Epoch 552/1000
38/38 - 0s - loss: 0.6382 - accuracy: 0.6471
Epoch 553/1000
38/38 - 0s - loss: 0.6382 - accuracy: 0.6455
Epoch 554/1000
38/38 - 0s - loss: 0.6377 - accuracy: 0.6529
Epoch 555/1000
38/38 - 0s - loss: 0.6362 - accuracy: 0.6504
Epoch 556/1000
38/38 - 0s - loss: 0.6450 - accuracy: 0.6496
Epoch 557/1000
38/38 - 0s - loss: 0.6410 - accuracy: 0.6421
Epoch 558/1000
38/38 - 0s - loss: 0.6398 - accuracy: 0.6455
Epoch 559/1000
38/38 - 0s - loss: 0.6377 - accuracy: 0.6479
Epoch 560/1000
38/38 - 0s - loss: 0.6408 - accuracy: 0.6455
Epoch 561/1000
38/38 - 0s - loss: 0.6437 - accuracy: 0.6372
Epoch 562/1000
38/38 - 0s - loss: 0.6354 - accuracy: 0.6488
Epoch 563/1000
38/38 - 0s - loss: 0.6385 - accuracy: 0.6562
Epoch 564/1000
38/38 - 0s - loss: 0.6400 - accuracy: 0.6504
Epoch 565/1000
38/38 - 0s - loss: 0.6343 - accuracy: 0.6496
Epoch 566/1000
38/38 - 0s - loss: 0.6354 - accuracy: 0.6446
Epoch 567/1000
38/38 - 0s - loss: 0.6386

Epoch 688/1000
38/38 - 0s - loss: 0.6320 - accuracy: 0.6504
Epoch 689/1000
38/38 - 0s - loss: 0.6334 - accuracy: 0.6537
Epoch 690/1000
38/38 - 0s - loss: 0.6376 - accuracy: 0.6405
Epoch 691/1000
38/38 - 0s - loss: 0.6351 - accuracy: 0.6463
Epoch 692/1000
38/38 - 0s - loss: 0.6314 - accuracy: 0.6620
Epoch 693/1000
38/38 - 0s - loss: 0.6331 - accuracy: 0.6430
Epoch 694/1000
38/38 - 0s - loss: 0.6306 - accuracy: 0.6504
Epoch 695/1000
38/38 - 0s - loss: 0.6291 - accuracy: 0.6512
Epoch 696/1000
38/38 - 0s - loss: 0.6320 - accuracy: 0.6512
Epoch 697/1000
38/38 - 0s - loss: 0.6310 - accuracy: 0.6545
Epoch 698/1000
38/38 - 0s - loss: 0.6305 - accuracy: 0.6537
Epoch 699/1000
38/38 - 0s - loss: 0.6313 - accuracy: 0.6545
Epoch 700/1000
38/38 - 0s - loss: 0.6329 - accuracy: 0.6537
Epoch 701/1000
38/38 - 0s - loss: 0.6352 - accuracy: 0.6545
Epoch 702/1000
38/38 - 0s - loss: 0.6388 - accuracy: 0.6570
Epoch 703/1000
38/38 - 0s - loss: 0.6342 - accuracy: 0.6521
Epoch 704/1000
38/38 - 0s - loss: 0.6332

Epoch 825/1000
38/38 - 0s - loss: 0.6313 - accuracy: 0.6537
Epoch 826/1000
38/38 - 0s - loss: 0.6246 - accuracy: 0.6570
Epoch 827/1000
38/38 - 0s - loss: 0.6313 - accuracy: 0.6603
Epoch 828/1000
38/38 - 0s - loss: 0.6255 - accuracy: 0.6579
Epoch 829/1000
38/38 - 0s - loss: 0.6200 - accuracy: 0.6678
Epoch 830/1000
38/38 - 0s - loss: 0.6281 - accuracy: 0.6612
Epoch 831/1000
38/38 - 0s - loss: 0.6231 - accuracy: 0.6628
Epoch 832/1000
38/38 - 0s - loss: 0.6257 - accuracy: 0.6562
Epoch 833/1000
38/38 - 0s - loss: 0.6255 - accuracy: 0.6570
Epoch 834/1000
38/38 - 0s - loss: 0.6299 - accuracy: 0.6603
Epoch 835/1000
38/38 - 0s - loss: 0.6242 - accuracy: 0.6603
Epoch 836/1000
38/38 - 0s - loss: 0.6242 - accuracy: 0.6595
Epoch 837/1000
38/38 - 0s - loss: 0.6249 - accuracy: 0.6545
Epoch 838/1000
38/38 - 0s - loss: 0.6246 - accuracy: 0.6570
Epoch 839/1000
38/38 - 0s - loss: 0.6321 - accuracy: 0.6612
Epoch 840/1000
38/38 - 0s - loss: 0.6277 - accuracy: 0.6686
Epoch 841/1000
38/38 - 0s - loss: 0.6281

Epoch 962/1000
38/38 - 0s - loss: 0.6281 - accuracy: 0.6537
Epoch 963/1000
38/38 - 0s - loss: 0.6223 - accuracy: 0.6579
Epoch 964/1000
38/38 - 0s - loss: 0.6254 - accuracy: 0.6620
Epoch 965/1000
38/38 - 0s - loss: 0.6199 - accuracy: 0.6653
Epoch 966/1000
38/38 - 0s - loss: 0.6219 - accuracy: 0.6620
Epoch 967/1000
38/38 - 0s - loss: 0.6210 - accuracy: 0.6686
Epoch 968/1000
38/38 - 0s - loss: 0.6246 - accuracy: 0.6686
Epoch 969/1000
38/38 - 0s - loss: 0.6201 - accuracy: 0.6554
Epoch 970/1000
38/38 - 0s - loss: 0.6191 - accuracy: 0.6554
Epoch 971/1000
38/38 - 0s - loss: 0.6184 - accuracy: 0.6620
Epoch 972/1000
38/38 - 0s - loss: 0.6225 - accuracy: 0.6620
Epoch 973/1000
38/38 - 0s - loss: 0.6233 - accuracy: 0.6628
Epoch 974/1000
38/38 - 0s - loss: 0.6221 - accuracy: 0.6562
Epoch 975/1000
38/38 - 0s - loss: 0.6176 - accuracy: 0.6686
Epoch 976/1000
38/38 - 0s - loss: 0.6226 - accuracy: 0.6620
Epoch 977/1000
38/38 - 0s - loss: 0.6218 - accuracy: 0.6653
Epoch 978/1000
38/38 - 0s - loss: 0.6182

In [23]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

24/24 - 0s - loss: 4.2885 - accuracy: 0.5332
Normal Neural Network - Loss: 4.2885003089904785, Accuracy: 0.5332446694374084
